<a href="https://colab.research.google.com/github/LIZZETHGOMEZ/Data-Science-Fase-3-BEDU-Santander/blob/main/Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Science Fase 3
## Módulo 4: Análisis de Datos con Python
## Team
- Omar Garduza
- Lizzeth Gómez Rodríguez
- Edgar Casanova
- Alan Sandoval


# NOTA: Hay que poner la descripción de las variables nuevamente

In [2]:
import pandas as pd
import numpy as np
from scipy import stats

In [3]:
# Cargamos los datos a utilizar (previamente procesados)
df = pd.read_csv("https://raw.githubusercontent.com/LIZZETHGOMEZ/Data-Science-Fase-3-BEDU-Santander/main/Data/cleanData.csv", index_col = 0)
df.head()

,country,year,respiratory_infections_and_tuberculosis_number,respiratory_infections_and_tuberculosis_percent,respiratory_infections_and_tuberculosis_rate,chronic_respiratory_diseases_number,chronic_respiratory_diseases_percent,chronic_respiratory_diseases_rate,all_greenhouse_gas,CO2,CH4,N2O,Fgas,mean_annual_temp,total_respiratory_diseases,total_diseases_percent
0,Mexico,1990,39290,0.08916,45.96035,16909,0.03839,19.78044,417.45,290.01,88.10,37.47,1.87,20.958333,56199,0.12755
1,Canada,1990,6797,0.03571,24.93967,9319,0.04895,34.19049,626.36,483.99,87.75,42.92,11.70,-7.450000,16116,0.08466
2,Israel,1990,772,0.02651,15.56473,1327,0.04556,26.75464,43.95,33.78,8.35,1.62,0.21,19.816667,2099,0.07207
3,Japan,1990,68816,0.08483,54.67434,29862,0.03681,23.72535,1109.37,1016.89,37.86,28.33,26.30,11.658333,98678,0.12164
4,Mexico,1991,37046,0.08447,42.53984,17171,0.03917,19.71738,435.45,308.41,88.71,36.36,1.97,20.891667,54217,0.12364


# NOTA
## Por qué el registro 4 es 1991 ?? Al parecer el df no está ordenado por años ni por países

## 1. Estimados de Locación y Variabilidad

Para una mejor descripción de los datos utilizaremos estadística descriptiva que nos permitirá conocer un panorama más detallado de los datos.

### Media

En principio nos gustaría conocer el valor típico de las muertes por enfermedades respiratorias en 1990 y en el año más reciente para saber cómo ha cambiado la cifra a lo largo del tiempo.
Y además conocer el valor típico de las muertes durante todo el periodo estudiado.

In [24]:
# Media del número de muertes totales por enfermedades respiratorias ( infeciones y tuberculosisi + crónicas)
mean_90 = df[df['year'] == 1990]['total_respiratory_diseases'].mean()
print(f'El promedio de las muertes por enfermedades respiratorias de los países de la OCDE en 1990 es de {mean_90:,.2f}')

mean_18 = df[df['year'] == 2018]['total_respiratory_diseases'].mean()
print(f'El promedio de las muertes por enfermedades respiratorias de los países de la OCDE en 2018 es de {mean_18:,.2f}')

mean_time = df['total_respiratory_diseases'].mean()
print(f'El promedio de las muertes por enfermedades respiratorias de los países de la OCDE durante el periodo 1990 -2018 es de {mean_time:,.2f}')

El promedio de las muertes por enfermedades respiratorias de los países de la OCDE en 1990 es de 18,863.88
El promedio de las muertes por enfermedades respiratorias de los países de la OCDE en 2018 es de 24,183.38
El promedio de las muertes por enfermedades respiratorias de los países de la OCDE durante el periodo 1990 -2018 es de 20,294.36


Notemos que de 1990 a 2018 las muertes por enfermedades respiratorias se han incrementado, veamos en que porcentaje a sido este incremento, para ello creamos una función que por practicidad podremos usar varias veces dentro de nuestro análisis.




In [5]:
# Porcentaje
def percentage(num_1,num_2):
  return num_2/num_1*100

incremento = percentage(mean_90, (mean_18-mean_90))

# Incremento de las muertes por enfermedades respiratorias
print(f'El promedio de las muertes por enfermedades respiratorias en los países de la OCDE se han incrementado en un {incremento:.2f}% durante el periodo de 1990 a 2018')


El promedio de las muertes por enfermedades respiratorias en los países de la OCDE se han incrementado en un 28.20% durante el periodo de 1990 a 2018


Observemos ahora los valores típicos para cada país en el periodo de estudio (1990-2018)


In [22]:
# Agrupamos por países
paises = df.groupby(['country'])

# Sumamos las muertes totales por enfermedades respiratorias de cada país durante el periodo 1990-2018
# y lo dividimos entre el total de años
round(paises['total_respiratory_diseases'].sum()/29, 2)

country
Australia          11232.07
Austria             4044.55
Belgium            12119.97
Canada             20468.45
Chile               9785.34
Denmark             6200.41
Estonia              632.00
Finland             3401.45
France             41030.79
Germany            57779.31
Greece              7618.93
Hungary             6537.93
Iceland              180.69
Ireland             4082.62
Israel              2698.38
Italy              36647.66
Japan             132896.72
Latvia              1100.07
Luxembourg           303.03
Mexico             50941.62
Netherlands        15179.48
New Zealand         2644.72
Norway              4604.00
Poland             21374.03
Portugal           11725.45
Slovakia            3136.55
Slovenia            1274.34
Spain              41097.76
Sweden              6923.31
Switzerland         4725.38
Turkey             42297.10
United Kingdom     84735.34
Name: total_respiratory_diseases, dtype: float64

Notemos que existen grandes diferencias entre los países respecto al promedio de todos en ese periodo que es de **20,294.36** ('mean_time') lo que quiere decir que los países presentan diferentes características y aplican diferentes políticas de salud y de combate al cambio climático, por lo que el impacto sobre la población de cada país es distinto.

Motivo por el cual es importante tener ambas perspectivas, una conjunta de todos los países para poder establecer una línea de referencia, otra que tome en cuenta los aspectos de cada país.

### Mediana



In [26]:
# Mediana
med_90 = df[df['year'] == 1990]['total_respiratory_diseases'].median()
print(f'La mediana de las muertes por enfermedades respiratorias de los países de la OCDE en 1990 es de {med_90:,.2f}')

med_18 = df[df['year'] == 2018]['total_respiratory_diseases'].median()
print(f'La mediana de las muertes por enfermedades respiratorias de los países de la OCDE en 2018 es de {med_18:,.2f}')

med_time = df['total_respiratory_diseases'].median()
print(f'La mediana de las muertes por enfermedades respiratorias de los países de la OCDE durante el periodo 1990-2018 es de {med_time:,.2f}')

La mediana de las muertes por enfermedades respiratorias de los países de la OCDE en 1990 es de 7,142.50
La mediana de las muertes por enfermedades respiratorias de los países de la OCDE en 2018 es de 9,291.50
La mediana de las muertes por enfermedades respiratorias de los países de la OCDE durante el periodo 1990-2018 es de 7,080.50


Observemos que la mediana está muy por debajo de los valores promedios antes encontrados, por lo que los datos podrían estar sesgados a la derecha, o quizá haya presencia de outliers.

Para entender mejor que podría estar pasando utilizaremos la media truncada, que nos permitte recortar un porcentaje de los valores extremos de nuestra muestra.

### Media Truncada